<a href="https://colab.research.google.com/github/URK21CO3003NathanShibuJohn/FinalYearProject/blob/main/FeatureSelectionNew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import scipy.signal as signal
from scipy.fftpack import fft
from scipy.signal import coherence
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_classif

# Load Preprocessed EEG Dataset
ndata = pd.read_csv("/content/drive/MyDrive/NData_Preprocessednew.csv")

# Feature Extraction Functions

def extract_time_features(eeg_signal):
    return np.mean(eeg_signal), np.std(eeg_signal), np.median(eeg_signal)

def extract_frequency_features(eeg_signal, fs=256):
    freq_spectrum = np.abs(fft(eeg_signal))
    power_spectral_density = np.sum(freq_spectrum ** 2) / len(freq_spectrum)
    return power_spectral_density

def extract_time_frequency_features(eeg_signal, fs=256, nperseg=64):
    f, t, Zxx = signal.stft(eeg_signal, fs=fs, nperseg=nperseg)
    return np.mean(np.abs(Zxx)), np.std(np.abs(Zxx))

def extract_spatial_features(eeg_data):
    num_channels = eeg_data.shape[1]
    coherence_values = []
    for i in range(num_channels):
        for j in range(i+1, num_channels):
            f, coh = coherence(eeg_data[:, i], eeg_data[:, j])
            coherence_values.append(np.mean(coh))
    return np.mean(coherence_values)

def extract_temporal_features(eeg_signal):
    first_derivative = np.diff(eeg_signal)
    trend = np.polyfit(np.arange(len(eeg_signal)), eeg_signal, 1)[0]
    return np.mean(first_derivative), np.std(first_derivative), trend

# Extract Features
features = []
labels = ndata['Label'].values
eeg_data = ndata.iloc[:, :-1].values

for i in range(eeg_data.shape[0]):
    time_features = extract_time_features(eeg_data[i, :])
    freq_features = extract_frequency_features(eeg_data[i, :])
    time_freq_features = extract_time_frequency_features(eeg_data[i, :])
    spatial_features = extract_spatial_features(eeg_data)
    temporal_features = extract_temporal_features(eeg_data[i, :])
    features.append(np.hstack([time_features, freq_features, time_freq_features, spatial_features, temporal_features]))

# Convert to DataFrame
feature_columns = ['Mean', 'StdDev', 'Median', 'PowerSpectralDensity', 'MeanSTFT', 'StdSTFT', 'Coherence', 'MeanDerivative', 'StdDerivative', 'Trend']
nfeatures = pd.DataFrame(features, columns=feature_columns)
nfeatures['Label'] = labels

# Adaptive Emotion-Specific Feature Selection (AEFS)
def adaptive_feature_selection(features, labels, num_features=5):
    relevance_scores = mutual_info_classif(features, labels)
    top_indices = np.argsort(relevance_scores)[-num_features:]
    return features.iloc[:, top_indices]

selected_features = adaptive_feature_selection(nfeatures.iloc[:, :-1], labels)
selected_features['Label'] = labels

# Save Selected Features
#selected_features.to_csv("/mnt/data/NData_Selected_Features.csv", index=False)

print("Feature selection complete. Data saved to 'NData_Selected_Features.csv'")


/usr/local/lib/python3.11/dist-packages/scipy/signal/_spectral_py.py:1240: UserWarning: nperseg = 64 is greater than input length  = 9, using nperseg = 9
  freqs, time, Zxx = _spectral_helper(x, x, fs, window, nperseg, noverlap,


Feature selection complete. Data saved to 'NData_Selected_Features.csv'


In [ ]:
selected_features.head()

,StdSTFT,MeanDerivative,StdDerivative,Trend,Coherence,Label
0,0.085732,0.024425,0.107404,0.018552,0.249071,Happy
1,0.094366,0.014327,0.083367,0.006112,0.249071,Happy
2,0.080155,0.012042,0.068483,0.013194,0.249071,Happy
3,0.070472,0.030834,0.084577,0.027888,0.249071,Happy
4,0.085266,0.018078,0.191948,-0.000047,0.249071,Happy


In [ ]:
selected_features.shape

(10000, 6)

In [ ]:
selected_features.to_csv("/content/drive/MyDrive/NData_Selected_FeaturesNew.csv", index=False)